In [1]:
import pandas as pd

## Набор данных

In [173]:
path_dataset = "D:/datasets/flickr-30k-images"
path_captions = "D:/YandexDisk/datasets/flickr-30k-images-metadata.csv"
path_prepared_captions = "D:/flickr-30k-images-metadata.csv"

In [47]:
df = pd.read_csv(path_captions, delimiter='|')
df.head(5)

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their...
1,1000092795.jpg,1,"Two young , White males are outside near many..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard .
3,1000092795.jpg,3,A man in a blue shirt standing in a garden .
4,1000092795.jpg,4,Two friends enjoy time spent together .


In [98]:
df.iat[1, 2]

'Two young, white males are outside near many bushes'

# Подготовка к переводу

### Исправить некоторые знаки препинания

– Первая буква предложения – заглавная.
– Удалить пробелы в начале и в конце.
– Удалить точку в конце.
– Удалить точку.
– Удалить пробел перед запятой.
– Удалить пробел перед апострофом.
– Удалить многоточие.
– Удалить вопросительный знак.
– Удалить восклицательный знак.
– Заменить двоеточие на запятую.
– Заменить точку с запятой на запятую.
– Заменить латинский апостроф на английский.

In [2]:
def fix_punctuation_marks(a_sentence):
    return a_sentence\
        .strip()\
        .capitalize()\
        .replace(" .", "")\
        .replace(".", "")\
        .replace(" ,", ",")\
        .replace(" '", "'")\
        .replace(" ...", "")\
        .replace("...", "")\
        .replace(" ?", "")\
        .replace("?", "")\
        .replace(" !", "")\
        .replace("!", "")\
        .replace(" : ", ", ")\
        .replace(" :", ", ")\
        .replace(" ; ", ", ")\
        .replace("’", "'")

### Заменить некоторые числа на текст

In [43]:
def replace_some_numbers_with_text(a_sentence):
    return a_sentence\
        .replace(" 0 ", "zero")\
        .replace(" 1 ", "one")\
        .replace(" 2 ", "two")\
        .replace(" 3 ", "three")\
        .replace(" 4 ", "four")\
        .replace(" 5 ", "five")\
        .replace(" 6 ", "six")\
        .replace(" 7 ", "seven")\
        .replace(" 8 ", "eight")\
        .replace(" 9 ", "nine")

### Заменить некоторые числа в начале предложения на текст

In [4]:
def replace_some_start_numbers_with_text(a_sentence):
    if a_sentence[1] != ' ':
        return a_sentence

    text = ""
    if a_sentence[0] == '0':
        text = "Zero"
    elif a_sentence[0] == '1':
        text = "One"
    elif a_sentence[0] == '2':
        text = "Two"
    elif a_sentence[0] == '3':
        text = "Three"
    elif a_sentence[0] == '4':
        text = "Four"
    elif a_sentence[0] == '5':
        text = "Five"
    elif a_sentence[0] == '6':
        text = "Six"
    elif a_sentence[0] == '7':
        text = "Seven"
    elif a_sentence[0] == '8':
        text = "Eight"
    elif a_sentence[0] == '9':
        text = "Nine"

    a_sentence = text + a_sentence[1:]
    return a_sentence

# ***

– Найти кавычки.
– Найти числа.

In [10]:
def is_there_number(sentence):
    for i in sentence.split():
        if i.isdigit():
            return True

    return False

In [3]:
def fix_dataframe(data, out_quotation_idxs, out_numbers_idxs):
    count_chars = 0

    for idx in data.index:
        sentence = fix_punctuation_marks(data.iat[idx, 2])
        sentence = replace_some_numbers_with_text(sentence)
        sentence = replace_some_start_numbers_with_text(sentence)
        data.iat[idx, 2] = sentence

        if sentence.find('"') != -1:
            out_quotation_idxs.append(idx)

        if is_there_number(sentence):
            out_numbers_idxs.append(idx)

        count_chars += len(sentence)

    return count_chars

In [54]:
quotation_idxs = []
numbers_idxs = []
chars_num = fix_dataframe(df, quotation_idxs, numbers_idxs)

print(chars_num)

9871649


### Найти кавычки

In [6]:
some_idx = quotation_idxs[0]
df.iat[some_idx, 2]

'The " white out " conditions of snow on the ground seem to almost obliverate the details of a man dressed for the cold weather in a heavy jacket and red hat riding a bicycle in a suburban neighborhood'

### Найти числа

In [15]:
string = "I am 45 years old"
print(is_there_number(string))

True


In [143]:
some_idx = numbers_idxs[0]
df.iat[some_idx, 2]

'A man in an orange shirt sings for the crowd below a sign that says lucky 13'

## Удалить изображения, у которых в описании есть кавычки

In [7]:
df_with_quots = df.loc[quotation_idxs, :]
images_with_quots = df_with_quots["image_name"].unique()
images_with_quots.shape

(672,)

In [9]:
df = df[~df.image_name.isin(images_with_quots)]
df.shape

(155555, 3)

In [11]:
df.head(5)

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their ...
1,1000092795.jpg,1,"Two young, white males are outside near many b..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard
3,1000092795.jpg,3,A man in a blue shirt standing in a garden
4,1000092795.jpg,4,Two friends enjoy time spent together


## Удалить изображения, у которых в описании есть числа


In [108]:
df_captions_with_numbers = df_captions.loc[idx_with_numbers, :]
images_to_delete = df_captions_with_numbers["image_name"].unique()
images_to_delete.shape

(322,)

In [144]:
df_captions.shape

(155555, 3)

In [146]:
df_captions = df_captions[~df_captions.image_name.isin(images_to_delete)]
df_captions.shape

(153945, 3)

## Удалить изображения, у которых в описании есть время (двоеточие)

In [179]:
idx_with_semicolon = []

for idx in df_captions.index:
    find_idx = df_captions.iat[idx, 2].find(':')
    if find_idx != -1:
        idx_with_semicolon.append(idx)

len(idx_with_semicolon)

0

In [172]:
some_idx = idx_with_semicolon[0]
df_captions.iat[some_idx, 2]

"Celebrating st patrick's day at 2:10 pm & an police officer is going inside a room"

In [173]:
df_captions_with_semicolon = df_captions.loc[idx_with_semicolon, :]
images_to_delete = df_captions_with_semicolon["image_name"].unique()
images_to_delete.shape

(7,)

In [174]:
df_captions = df_captions[~df_captions.image_name.isin(images_to_delete)]
df_captions.shape

(153910, 3)

## Сохранить dataframe в файл

In [97]:
df_captions.to_csv(path_prepared_captions, sep='|', encoding='utf-8', index=False)